# Logistic Regression

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib

data = pd.read_csv("C:\\Users\\racha\\OneDrive\\Desktop\\training_data.csv")

X = data.drop(['Target', 'Time', 'Source', 'Destination', 'Protocol'], axis=1)
X = X.select_dtypes(include=['float64', 'int64'])
y = data['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions))

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

joblib.dump(model, 'ddos_model.pkl')

              precision    recall  f1-score   support

           0       1.00      0.68      0.81      2263
           1       0.77      1.00      0.87      2332

    accuracy                           0.84      4595
   macro avg       0.88      0.84      0.84      4595
weighted avg       0.88      0.84      0.84      4595

Accuracy: 0.8443960826985855


['ddos_model.pkl']

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib

data = pd.read_csv("C:\\Users\\racha\\OneDrive\\Desktop\\training_data.csv")

X = data.drop(['Target', 'Time', 'Source', 'Destination', 'Protocol'], axis=1)
X = X.select_dtypes(include=['float64', 'int64'])
y = data['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler object
joblib.dump(scaler, 'scaler.pkl')

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions))

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

joblib.dump(model, 'ddos_model1.pkl')


              precision    recall  f1-score   support

           0       1.00      0.68      0.81      2263
           1       0.77      1.00      0.87      2332

    accuracy                           0.84      4595
   macro avg       0.88      0.84      0.84      4595
weighted avg       0.88      0.84      0.84      4595

Accuracy: 0.8443960826985855


['ddos_model1.pkl']

In [3]:
from scapy.all import sniff, IP

def extract_features(packet):
    if IP in packet:
        src_ip = packet[IP].src
        dst_ip = packet[IP].dst
        packet_len = len(packet)
        return src_ip, dst_ip, packet_len
    return None

def process_packet(packet):
    features = extract_features(packet)
    if features:
        src_ip, dst_ip, packet_len = features
        print(f"Source IP: {src_ip}, Destination IP: {dst_ip}, Packet Length: {packet_len}")

sniff(prn=process_packet, count=10)


Source IP: 192.168.28.46, Destination IP: 13.89.178.26, Packet Length: 55
Source IP: 192.168.28.46, Destination IP: 35.174.127.31, Packet Length: 300
Source IP: 192.168.28.46, Destination IP: 35.174.127.31, Packet Length: 300
Source IP: 192.168.28.34, Destination IP: 239.255.255.250, Packet Length: 218
Source IP: 35.174.127.31, Destination IP: 192.168.28.46, Packet Length: 54
Source IP: 35.174.127.31, Destination IP: 192.168.28.46, Packet Length: 321
Source IP: 35.174.127.31, Destination IP: 192.168.28.46, Packet Length: 66
Source IP: 192.168.28.46, Destination IP: 35.174.127.31, Packet Length: 54
Source IP: 192.168.28.34, Destination IP: 239.255.255.250, Packet Length: 218
Source IP: 192.168.28.34, Destination IP: 239.255.255.250, Packet Length: 218


<Sniffed: TCP:7 UDP:3 ICMP:0 Other:0>

In [ ]:
from scapy.all import sniff
from scapy.layers.inet import IP, TCP, UDP
from datetime import datetime

def extract_packet_info(packet):
    if IP in packet and TCP in packet:
        src_ip = packet[IP].src
        dst_ip = packet[IP].dst
        src_port = packet[TCP].sport
        dst_port = packet[TCP].dport
        protocol = 'tcp'
        packet_len = len(packet)
        timestamp = datetime.now().strftime("%H:%M.%S")
        return src_ip, dst_ip, src_port, dst_port, packet_len, protocol, timestamp
    return None, None, None, None, None, None, None

def packet_callback(packet):
    src_ip, dst_ip, src_port, dst_port, packet_len, protocol,timestamp = extract_packet_info(packet)
    if src_ip:
        #print(f"Source IP: {src_ip}, Destination IP: {dst_ip}, Source Port: {src_port}, Destination Port: {dst_port}, Packet Length: {packet_len}, Protocol: {protocol}")
        with open('captured_traffic.csv', 'a') as file:
            file.write(f"{timestamp},{src_ip},{dst_ip},{protocol},{packet_len},{src_port},{dst_port}\n")

with open('captured_traffic_scapy.csv', 'w') as file:
    file.write("Time,Source,Destination,Protocol,Length,Src port,Dest port\n")

start_time = time.time()
while time.time() - start_time < 50:
    sniff(prn=packet_callback, count=10000)


# Testing with live data csv

In [ ]:
import pandas as pd
import joblib
data = pd.read_csv("captured_traffic0.csv")
X = data.drop(['Time', 'Source', 'Destination', 'Protocol'], axis=1)
X = X.select_dtypes(include=['float64', 'int64'])
model = joblib.load('ddos_model.pkl')
predictions = model.predict(X)
predictions = model.predict(X)
data['prediction'] = predictions
data.to_csv('captured_traffic_with_predictions.csv', index=False)